# Draft

Author's notes: 
- Remove Introduction headings. Replace it with just the opening.

## Introduction

Data tells stories that numbers alone cannot. In the context of the Philippines, where terrorism has left its mark on communities and shaped security policies, understanding the trends behind these incidents is vital. By diving into the data, we can uncover patterns, identify hotspots, and provide actionable insights to help address these challenges effectively.